In [1]:
!pip install pandas scikit-learn sentence_transformers python-docx bitsandbytes accelerate transformers protobuf sentencepiece openpyxl

In [4]:
import os
import pandas as pd
from docx import Document
from transformers import AutoTokenizer, MPNetModel, AutoModelForCausalLM, BitsAndBytesConfig
from sklearn.metrics.pairwise import cosine_similarity
import torch
import re
import numpy as np
from sentence_transformers import SentenceTransformer
import time

# Загрузка моделей для работы с эмбеддингами
print("Загрузка модели MPNet для эмбеддингов...")
tokenizer = AutoTokenizer.from_pretrained("microsoft/mpnet-base")
model = MPNetModel.from_pretrained("microsoft/mpnet-base")
sts_model = SentenceTransformer('all-MiniLM-L6-v2')

# Настройка для LLM модели, которая квантована в 4-бит для скорости
print("Загрузка LLM модели...")

hf_token = "<ВАШ ТОКЕН HUGGING FACE>" #Нужно для загрузки Mistral. Перед созданием токена нужно получить доступ к модели тут https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2




model_name = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
llm_tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)

if llm_tokenizer.pad_token_id is None:
    llm_tokenizer.pad_token_id = llm_tokenizer.eos_token_id

llm_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    quantization_config=bnb_config,
    device_map={"": "cuda:0"},
    torch_dtype=torch.float16,
)

print("Модели загружены.")


Загрузка модели MPNet для эмбеддингов...


Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.bias', 'mpnet.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Загрузка LLM модели...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Модели загружены.


In [9]:
# Функция для извлечения текста из docx файла
def extract_text_from_docx(filepath):
    """
    Извлекает полный текст из .docx файла и объединяет его в одну строку.
    Возвращает весь текст документа в виде строки, где каждый параграф отделен переносом строки.
    """
    doc = Document(filepath)
    return "\n".join([para.text for para in doc.paragraphs])

# Функция для генерации эмбеддинга текста с использованием модели MPNet
def get_text_embedding(text):
    """
    Генерирует эмбеддинг для заданного текста, используя модель MPNet.
    Текст сначала токенизируется и обрабатывается моделью, затем возвращается среднее значение эмбеддингов по всем токенам.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze()


def get_sts_embedding(text):
    """
    Генерирует эмбеддинг для текста с использованием модели SentenceTransformer.
    Возвращает тензор, представляющий векторное представление текста.
    """
    return torch.tensor(sts_model.encode(text, convert_to_tensor=False))

def extract_text_structured(file_path):
    """
    Извлекает и структурирует текст из .docx файла. 
    Разделяет текст на секции, основываясь на форматировании документа (напр., нумерации и заголовках).
    Возвращает словарь, где ключи — это имена секций, а значения — их содержимое.
    """
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    section_positions = [(match.start(), match.group().strip()) for match in re.finditer(r"(.*?[:])|([0-9]+\.)", text)]
    structured_text = {}
    for idx, (start_pos, section_name) in enumerate(section_positions):
        end_pos = section_positions[idx + 1][0] if idx + 1 < len(section_positions) else len(text)
        section_content = text[start_pos:end_pos].strip()
        structured_text[section_name.lower().strip(":")] = section_content
    return structured_text

def match_sections_with_threshold(text_uc, text_ssts, threshold=0.7):
    """
    Сопоставляет секции UC и SSTS документов, используя эмбеддинги и заданный порог схожести.
    Возвращает словарь с сопоставленными секциями и уровнем схожести для каждой из них.
    """
    matched_sections = {}
    uc_content_embeddings = {section: get_text_embedding(content) for section, content in text_uc.items()}
    uc_content_sts_embeddings = {section: get_sts_embedding(content) for section, content in text_uc.items()}
    
    for uc_section, uc_embedding in uc_content_embeddings.items():
        similarities = []
        
        for ssts_section, ssts_content in text_ssts.items():
            ssts_embedding = get_text_embedding(ssts_content)
            ssts_sts_embedding = get_sts_embedding(ssts_content)
            
            # Проверяем размерность эмбеддингов, если совпадает — считаем схожесть
            if uc_embedding.dim() == ssts_embedding.dim():
                cosine_sim = cosine_similarity(
                    uc_embedding.unsqueeze(0).numpy(),
                    ssts_embedding.unsqueeze(0).numpy()
                ).item()
            else:
                cosine_sim = 0
            
            # Считаем косинусное сходство с помощью SentenceTransformer
            sts_sim = torch.nn.functional.cosine_similarity(
                uc_content_sts_embeddings[uc_section].unsqueeze(0),
                ssts_sts_embedding.unsqueeze(0),
                dim=1
            ).item()
            
            combined_sim = (cosine_sim + sts_sim) / 2
            similarities.append((ssts_section, combined_sim))
        
        # Находим лучшее совпадение с наибольшей схожестью
        best_match = max(similarities, key=lambda x: x[1], default=(None, 0))
        
        if best_match[1] > threshold:
            matched_sections[uc_section] = {
                'matched_section': best_match[0],
                'content_uc': text_uc[uc_section],
                'content_ssts': text_ssts.get(best_match[0], "No content"),
                'similarity': best_match[1]
            }
        else:
            matched_sections[uc_section] = {
                'matched_section': None,
                'content_uc': text_uc[uc_section],
                'content_ssts': None,
                'similarity': best_match[1]
            }

    return matched_sections

def compute_similarity_metrics(matched_sections):
    """
    Вычисляет метрики сходства, такие как среднее, медианное и стандартное отклонение.
    Возвращает словарь с рассчитанными статистическими метриками для переданных значений сходства.
    """
    similarities = [
        info['similarity'] for info in matched_sections.values()
        if isinstance(info, dict) and 'similarity' in info
    ]
    return {
        "mean_similarity": np.mean(similarities),
        "median_similarity": np.median(similarities),
        "std_deviation": np.std(similarities),
        "min_similarity": np.min(similarities),
        "max_similarity": np.max(similarities)
    }

def evaluate_compliance(metrics):
    """
    Оценивает соответствие на основе метрик сходства.
    Возвращает уровень соответствия (например, 'FC' для полного соответствия или 'NC' для отсутствия соответствия) 
    на основе заданных порогов для метрик.
    """
    mean_similarity = metrics["mean_similarity"]
    median_similarity = metrics["median_similarity"]
    min_similarity = metrics["min_similarity"]
    std_deviation = metrics["std_deviation"]
    
    if mean_similarity >= 0.6 and median_similarity >= 0.6 and min_similarity >= 0.6 and std_deviation <= 0.1:
        return "FC"
    elif mean_similarity >= 0.6 and median_similarity >= 0.6 and min_similarity >= 0.35 and std_deviation <= 0.2:
        return "LC"
    elif mean_similarity >= 0.55 and median_similarity >= 0.55 and min_similarity >= 0.3 and std_deviation <= 0.25:
        return "PC"
    else:
        return "NC"

def generate_difference_text(uc_text, ssts_text):
    prompt = f"""You are a comparison model designed to analyze technical specifications from UC and SSTS documents. Your task is to identify only the most important differences in the functional details described in the SSTS Specification compared to the UC Specification.
    UC Specification:
    [{uc_text}]

    SSTS Specification:
    [{ssts_text}]

    Provide a very short list of the most important differences where SSTS misses certain details or differs in functionality from UC.
    If there are no functional differences, answer "-"
    Your answer never exceeds 5 sentences.
    List the differences found:
"""
    inputs = llm_tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
    inputs = {key: value.to("cuda:0") for key, value in inputs.items()}
    generated_ids = llm_model.generate(**inputs, max_new_tokens=450, do_sample=True)
    decoded = llm_tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    if "List the differences found:" in decoded:
        return decoded.split("List the differences found:")[-1].strip()
    return "-"

# Доки вместе с файлом
directory = 'train_data'
results = []
times = []  # Для хранения времени обработки каждого файла

# Измененный код для извлечения `name`
for uc_filename in os.listdir(directory):
    if uc_filename.startswith("UC-") and uc_filename.endswith(".docx"):
        ssts_filename = uc_filename.replace("UC-", "SSTS-")
        ssts_filepath = os.path.join(directory, ssts_filename)
        uc_filepath = os.path.join(directory, uc_filename)
        
        start_time = time.time()  # Начало замера времени для файла

        # Проверка на наличие файла SSTS
        if os.path.exists(ssts_filepath):
            # Извлечение и обработка текста
            text_uc = extract_text_structured(uc_filepath)
            text_ssts = extract_text_structured(ssts_filepath)
            matched_sections = match_sections_with_threshold(text_uc, text_ssts)
            metrics = compute_similarity_metrics(matched_sections)
            compliance = evaluate_compliance(metrics)
            
            # Генерация описания различий
            uc_text = extract_text_from_docx(uc_filepath)
            ssts_text = extract_text_from_docx(ssts_filepath)
            differences = generate_difference_text(uc_text, ssts_text)

            # Извлечение `name` из содержимого UC-файла с проверкой формата
            name_line = uc_text.splitlines()[0]  # Извлечение первой строки
            if re.match(r'^\[.*?\]', name_line):  # Проверка, есть ли идентификатор в квадратных скобках
                name = re.sub(r'^\[.*?\]\s*', '', name_line).strip()  # Удаление идентификатора
            else:
                name = number  # Если идентификатора нет, используем `number`

            # Добавление в результаты
            number = uc_filename.split("-")[1].split(".")[0]
            results.append({
                "Number": number,
                "Name": name,
                "Differences": differences,
                "Description": "",  
                "Complience Level": compliance
            })

        else:
            # Если пара SSTS не найдена
            number = uc_filename.split("-")[1].split(".")[0]
            results.append({
                "Number": number,
                "Name": number,  # Используем `number` в качестве `name`
                "Differences": "",
                "Description": "",
                "Complience Level": "NA"
            })
            print('Нет соответствующего ssts файла')



        # Конец замера времени для файла
        end_time = time.time()
        file_time = end_time - start_time
        times.append(file_time)  # Сохранение времени обработки
        print(f'Проверка для uc-{number} Уровень: {compliance}')
        print(f"Время обработки для пары {uc_filename} и {ssts_filename}: {file_time:.2f} секунд")


# Подсчет и вывод среднего времени обработки
average_time_per_file = np.mean(times)
print(f"Среднее время обработки одного файла: {average_time_per_file:.2f} секунд")

# Создание CSV-файла
results_df = pd.DataFrame(results)
results_df.to_csv('submission_format.csv', index=False)
print("CSV-файл 'submission_format.csv' создан.")

# Создание Excel-файла
results_df.to_excel('submission_format.xlsx', index=False)
print("Excel-файл 'submission_format.xlsx' создан.")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Проверка для uc-28561 Уровень: LC
Время обработки для пары UC-28561.docx и SSTS-28561.docx: 12.90 секунд


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Проверка для uc-31523 Уровень: LC
Время обработки для пары UC-31523.docx и SSTS-31523.docx: 9.60 секунд


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Проверка для uc-8692 Уровень: LC
Время обработки для пары UC-8692.docx и SSTS-8692.docx: 17.38 секунд


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Проверка для uc-30371 Уровень: LC
Время обработки для пары UC-30371.docx и SSTS-30371.docx: 20.64 секунд


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Проверка для uc-6583 Уровень: LC
Время обработки для пары UC-6583.docx и SSTS-6583.docx: 15.12 секунд
Нет соответствующего ssts файла
Проверка для uc-26160 Уровень: LC
Время обработки для пары UC-26160.docx и SSTS-26160.docx: 0.00 секунд
Среднее время обработки одного файла: 12.34 секунд
CSV-файл 'submission_format.csv' создан.
Excel-файл 'submission_format.xlsx' создан.
